# Tugas 1 Crawling PTA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
!pip install requests
!pip install html5lib
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=0c5d230f57c774a7a17f7fdc353d830dfae7ace451fcedce8a91bf1391c9f56e
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
# constant
LAST_INDEX = -1
FIRST_INDEX = 1
INCREMENT_BY_ONE = 1

In [ ]:
# import library
from bs4 import BeautifulSoup
import requests

def request_url(url):
    return requests.get(url)

def request_header_url(header, url_website):
    headers = {'User-Agent': header}
    return requests.get(url=url_website, headers=headers)

def parse_website(request):
    """Use html5lib library to parse"""
    return BeautifulSoup(request.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib

def prettify_web_structure(parsed_page):
    parsed_page.prettify()

def get_content_table(web_element, tag, attributes):
    return web_element.find(tag, attrs = attributes)

In [ ]:
# max_pages = 172 # total halaman di website PTA, terbaru februari 2023

r = request_header_url("Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1", "https://pta.trunojoyo.ac.id/c_search/byprod/10")
soup = parse_website(r)
prettify_web_structure(soup)
table = get_content_table(soup, "div", {"id":"wrapper"})

pagination = table.findAll("a", attrs = {"class":"pag_button"})
total_pages = int(pagination[LAST_INDEX]["href"].split("/")[LAST_INDEX])

papers = []
for pages in range(FIRST_INDEX, total_pages + INCREMENT_BY_ONE):

    url_link = "https://pta.trunojoyo.ac.id/c_search/byprod/10"
    id_prodi = url_link.split("/")[LAST_INDEX]
    nama_prodi = ""
    if (id_prodi == "10"):
        nama_prodi = "Teknik Informatika"

    r_pages = request_header_url("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", f"{url_link}/{pages}")
    soup_pages = parse_website(r_pages)
    prettify_web_structure(soup_pages)
    table_pages = get_content_table(soup_pages, "div", {"id":"wrapper"})

    for article_row in table_pages.findAll("a", attrs = {"class":"gray button"}):
        r_article_row = request_header_url("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", article_row["href"])
        soup_article_row = parse_website(r_article_row)
        prettify_web_structure(soup_article_row)
        table_article_row = get_content_table(soup_article_row, "li", {"data-id":"id-1", "data-cat":"#luxury"})
        article_row_title = table_article_row.find('a', attrs = {"class":"title", "href":"#"}).string
        article_row_penulis = ""
        article_row_pembimbing1 = ""
        article_row_pembimbing2 = ""
        article_row_abstract = table_article_row.find('p', attrs = {"align":"justify"}).string
        spans = table_article_row.find_all('span')
        for span in spans:
          span_text = span.get_text(strip=True)
          if "Penulis :" in span_text:
            article_row_penulis = span_text.replace("Penulis :", "").strip()
          elif "Dosen Pembimbing I :" in span_text:
            article_row_pembimbing1 = span_text.replace("Dosen Pembimbing I :", "").strip()
          elif "Dosen Pembimbing II :" in span_text:
            article_row_pembimbing2 = span_text.replace("Dosen Pembimbing II :", "").strip()

        papers.append([
                article_row_title,
                article_row_penulis,
                article_row_pembimbing1,
                article_row_pembimbing2,
                article_row_abstract
        ])

In [ ]:
df = pd.DataFrame(papers, columns=["Judul", "Penulis", "pembimbing I", "pembimbing II", "Abstrak"])
df

,Judul,Penulis,pembimbing I,pembimbing II,Abstrak
0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,A.Ubaidillah S.Kom,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (SIAKAD) merupaka...
1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"M. Basith Ardianto,","Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...
2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,"Akhmad Suyandi, S.Kom","Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...
3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Heri Supriyanto,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di Perguruan Tinggi me...
4,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Septian Rahman Hakim,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...
...,...,...,...,...,...
853,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Rachmad Agung Pambudi,"Eka Mala Sari Rochman, S.Kom., M.Kom","Sri Herawati, S.Kom., M.Kom",Investasi saham selama ini memiliki resiko ker...
854,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Nadila Hidayanti,"Achmad Jauhari, S.T., M.Kom","Ika Oktavia Suzanti, S.Kom., M.Cs",Information Retrieval (IR) merupakan pengambil...
855,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Afni Sakinah,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Moch. Kautsar Sophan, S.Kom., M.MT.",Klasifikasi citra merupakan proses pengelompok...
856,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Friska Fatmawatiningrum,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Prof. Dr. Arief Muntasa, S.Si., M.MT.",Identifikasi atribut pejalan kaki merupakan sa...


In [ ]:
import os
df.to_csv('/content/drive/MyDrive/PPW/Tugas/PTA.csv')
# pd.read_csv(/content/drive/MyDrive/PPW/Tugas/PTA.csv')

# Tugas 2 Normalisasi Text



1.  Punctuation process menghilangkan tanda baca dan simbol
2.  stopword
1.  Tokenisasi
2.  Steeming
1.  Feature extraction dan membentuk VSM dalam term frequency, logarithm freqency, one-hot encoding,  TF-IDF

### Install dan Import Modul atau Library

In [ ]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import warnings
import pandas as pd
import numpy as np
import nltk
import re
import csv

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Load Data

In [ ]:
data_x = pd.read_csv('/content/drive/MyDrive/PPW/Tugas/PTA.csv')
data_x

,Unnamed: 0,Judul,Penulis,pembimbing I,pembimbing II,Abstrak
0,0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,A.Ubaidillah S.Kom,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (SIAKAD) merupaka...
1,1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"M. Basith Ardianto,","Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...
2,2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,"Akhmad Suyandi, S.Kom","Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...
3,3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Heri Supriyanto,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di Perguruan Tinggi me...
4,4,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Septian Rahman Hakim,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...
...,...,...,...,...,...,...
853,853,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Rachmad Agung Pambudi,"Eka Mala Sari Rochman, S.Kom., M.Kom","Sri Herawati, S.Kom., M.Kom",Investasi saham selama ini memiliki resiko ker...
854,854,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Nadila Hidayanti,"Achmad Jauhari, S.T., M.Kom","Ika Oktavia Suzanti, S.Kom., M.Cs",Information Retrieval (IR) merupakan pengambil...
855,855,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Afni Sakinah,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Moch. Kautsar Sophan, S.Kom., M.MT.",Klasifikasi citra merupakan proses pengelompok...
856,856,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Friska Fatmawatiningrum,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Prof. Dr. Arief Muntasa, S.Si., M.MT.",Identifikasi atribut pejalan kaki merupakan sa...


In [ ]:
data_x.isnull().sum()

Unnamed: 0        0
Judul             6
Penulis          10
pembimbing I     10
pembimbing II    11
Abstrak          30
dtype: int64

In [ ]:
data_x = data_x.dropna()
data_x.isnull().sum()

Unnamed: 0       0
Judul            0
Penulis          0
pembimbing I     0
pembimbing II    0
Abstrak          0
dtype: int64

##Cleaning

**Menghapus Simbol, Tanda Baca, dan Karakter Tertentu**

In [ ]:
def cleaning(text):
  text = re.sub(r'[^a-zA-Z\s]', '', text).strip()
  return text

data_x['Cleaning'] = data_x['Judul'].apply(cleaning)
data_x['Cleaning']

0      PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...
1      APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...
2      RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...
3      SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...
4      SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...
                             ...                        
853    PENERAPAN ALGORITMA LONGSHORT TERM MEMORY UNTU...
854    SISTEM PENCARIAN TEKS ALQURAN TERJEMAHAN BERBA...
855    KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...
856    IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...
857    DETEKSI OBJEK MANUSIA BERBASIS ONE STAGE DETEC...
Name: Cleaning, Length: 827, dtype: object

**Tokenizing**

In [ ]:
def tokenizer(text):
  text = text.lower()
  return word_tokenize(text)

data_x['Tokenizing'] = data_x['Cleaning'].apply(tokenizer)
data_x['Tokenizing']

0      [perancangan, dan, implementasi, sistem, datab...
1      [aplikasi, kontrol, dan, monitoring, jaringan,...
2      [rancang, bangun, aplikasi, proxy, server, unt...
3      [sistem, pendukung, keputusan, optimasi, penja...
4      [sistem, augmented, reality, animasi, benda, b...
                             ...                        
853    [penerapan, algoritma, longshort, term, memory...
854    [sistem, pencarian, teks, alquran, terjemahan,...
855    [klasifikasi, kompleksitas, visual, citra, sam...
856    [identifikasi, biner, atribut, pejalan, kaki, ...
857    [deteksi, objek, manusia, berbasis, one, stage...
Name: Tokenizing, Length: 827, dtype: object

**Stopwords**

In [ ]:
corpus = stopwords.words('indonesian')

def stopwordText(words):
 return [word for word in words if word not in corpus]

data_x['Stopword Removal'] = data_x['Tokenizing'].apply(stopwordText)

# Gabungkan kembali token menjadi kalimat utuh
data_x['Full Text'] = data_x['Stopword Removal'].apply(lambda x: ' '.join(x))
data_x['Full Text']

0      perancangan implementasi sistem database terdi...
1      aplikasi kontrol monitoring jaringan komputer ...
2      rancang bangun aplikasi proxy server enkripsi ...
3      sistem pendukung keputusan optimasi penjadwala...
4      sistem augmented reality animasi benda bergera...
                             ...                        
853    penerapan algoritma longshort term memory pred...
854    sistem pencarian teks alquran terjemahan berba...
855    klasifikasi kompleksitas visual citra sampah k...
856    identifikasi biner atribut pejalan kaki adapti...
857    deteksi objek manusia berbasis one stage detec...
Name: Full Text, Length: 827, dtype: object

In [ ]:
def stemmingText(dokumens):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()

  return [stemmer.stem(i) for i in dokumens]

data_x['Stemming'] = data_x['Stopword Removal'].apply(stemmingText)
data_x['Stemming']

0      [ancang, implementasi, sistem, database, distr...
1      [aplikasi, kontrol, monitoring, jaring, komput...
2      [rancang, bangun, aplikasi, proxy, server, enk...
3      [sistem, dukung, putus, optimasi, jadwal, mata...
4      [sistem, augmented, reality, animasi, benda, g...
                             ...                        
853    [terap, algoritma, longshort, term, memory, pr...
854    [sistem, cari, teks, alquran, terjemah, bahasa...
855    [klasifikasi, kompleksitas, visual, citra, sam...
856    [identifikasi, biner, atribut, pejal, kaki, ad...
857    [deteksi, objek, manusia, bas, one, stage, det...
Name: Stemming, Length: 827, dtype: object

## VSM (Vector Space Model)

**One Hot Encoding**

In [ ]:
def pandasOneHotEncoder(dokumens):
  encoder  = pd.get_dummies(dokumens.apply(pd.Series).stack()).sum(level=0)
  data_x = pd.concat([dokumens, encoder], axis=1)

  return data_x

oneHotEncoder = pandasOneHotEncoder(data_x['Stopword Removal'])
oneHotEncoder

,Stopword Removal,a,abadi,abjad,abstrak,abstraksi,accelerometer,acceptance,account,accounting,...,wp,x,yayasan,yolo,yolov,you,zachman,zakat,zoning,zucara
0,"[perancangan, implementasi, sistem, database, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[aplikasi, kontrol, monitoring, jaringan, komp...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[rancang, bangun, aplikasi, proxy, server, enk...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[sistem, pendukung, keputusan, optimasi, penja...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"[sistem, augmented, reality, animasi, benda, b...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,"[penerapan, algoritma, longshort, term, memory...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
854,"[sistem, pencarian, teks, alquran, terjemahan,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
855,"[klasifikasi, kompleksitas, visual, citra, sam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
856,"[identifikasi, biner, atribut, pejalan, kaki, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
oneHotEncoder.to_csv('OneHotEncoder.csv', index=False)

**TF-IDF**

In [ ]:
def tfidf(dokumen):
  vectorizer = TfidfVectorizer()
  x = vectorizer.fit_transform(dokumen).toarray()
  terms = vectorizer.get_feature_names_out()

  tf_idf = pd.DataFrame(x, columns=terms)
  tf_idf.insert(0, 'Dokumen', dokumen)

  return (vectorizer, tf_idf)

tfidf_vectorizer, tf_idf = tfidf(data_x['Full Text'])
tf_idf

,Dokumen,abadi,abjad,abstrak,abstraksi,accelerometer,acceptance,account,accounting,activity,...,workflow,wp,yayasan,yolo,yolov,you,zachman,zakat,zoning,zucara
0,perancangan implementasi sistem database terdi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0
1,aplikasi kontrol monitoring jaringan komputer ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0
2,rancang bangun aplikasi proxy server enkripsi ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0
3,sistem pendukung keputusan optimasi penjadwala...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0
4,sistem augmented reality animasi benda bergera...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,sistem pendukung keputusan pemilihan pemain fu...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0
823,sistem pakar diagnosa penyakit saluran pencern...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0
824,implementasi teknik weighted knearest neighbor...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0
825,metode technique for others reference by simil...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0


In [ ]:
tf_idf.to_csv('TF-IDF.csv', index=False)

**Term Frekuensi**

In [ ]:
def term_freq(dokumens):
  # Buat objek CountVectorizer
  vectorizer = CountVectorizer()
  tf_matrix = vectorizer.fit_transform(dokumens).toarray()
  terms = vectorizer.get_feature_names_out()

  tf = pd.DataFrame(tf_matrix, columns=terms)
  tf.insert(0, 'Dokumen', dokumens)

  return (vectorizer, tf, tf_matrix, terms)

tf_vectorizer, tf, tf_matrix, tf_terms = term_freq(data_x['Full Text'])
tf

,Dokumen,abadi,abjad,abstrak,abstraksi,accelerometer,acceptance,account,accounting,activity,...,workflow,wp,yayasan,yolo,yolov,you,zachman,zakat,zoning,zucara
0,perancangan implementasi sistem database terdi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,aplikasi kontrol monitoring jaringan komputer ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,rancang bangun aplikasi proxy server enkripsi ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,sistem pendukung keputusan optimasi penjadwala...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,sistem augmented reality animasi benda bergera...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,sistem pendukung keputusan pemilihan pemain fu...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
823,sistem pakar diagnosa penyakit saluran pencern...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
824,implementasi teknik weighted knearest neighbor...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
825,metode technique for others reference by simil...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
tf.to_csv('Term Frequensi.csv', index=False)

**Logarithm Frekuensi**

In [ ]:
def log_freq(dokumens):
  return np.log10(dokumens + 1)

data_log_freq = pd.DataFrame(tf_matrix, columns=tf_terms).apply(log_freq)
data_log_freq.insert(0, 'Dokumen', data_x['Full Text'])
data_log_freq

,Dokumen,abadi,abjad,abstrak,abstraksi,accelerometer,acceptance,account,accounting,activity,...,workflow,wp,yayasan,yolo,yolov,you,zachman,zakat,zoning,zucara
0,perancangan implementasi sistem database terdi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0
1,aplikasi kontrol monitoring jaringan komputer ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0
2,rancang bangun aplikasi proxy server enkripsi ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0
3,sistem pendukung keputusan optimasi penjadwala...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0
4,sistem augmented reality animasi benda bergera...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,sistem pendukung keputusan pemilihan pemain fu...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0
823,sistem pakar diagnosa penyakit saluran pencern...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0
824,implementasi teknik weighted knearest neighbor...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0
825,metode technique for others reference by simil...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0


In [ ]:
data_log_freq.to_csv('Logarithm Frequensi.csv', index=False)